#### Inputting the required Libraries

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, time
import pytz
import ta
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import requests
from datetime import timedelta

#### Inputing the market data from Yahoo Finance

In [75]:
ticker = 'ADANIPOWER.NS'
df = yf.download(ticker, period="2y")

if isinstance(df.columns, pd.MultiIndex): # Flattening multi level columns
    df.columns = df.columns.get_level_values(0)

df['Close'] = df['Close'].squeeze() # Ensure Close is 1D Series
df['Volume'] = df['Volume'].squeeze()


df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
df.dropna(inplace=True)

/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_82860/3049154687.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y")
[*********************100%***********************]  1 of 1 completed


#### Deriving the values for the Technical Indicators from the data

In [76]:
df['rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
macd = ta.trend.MACD(df['Close'])
df['macd'] = macd.macd()
df['volatility'] = df['log_return'].rolling(12).std()
df['vol_chg'] = df['Volume'].pct_change()

df.dropna(inplace=True)

#### Setting Up FinBERT

In [77]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
model.eval()

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

#### TimeZone Handling for the news

In [78]:
IST = pytz.timezone("Asia/Kolkata")
MARKET_CLOSE = time(15, 30)

def map_to_trading_day(ts):
    if ts.weekday() >= 5:
        ts += pd.offsets.BDay()
    if ts.time() > MARKET_CLOSE:
        ts += pd.offsets.BDay()
    return pd.Timestamp(ts.date())

#### Fetching the news from NEWS Api

In [79]:
NEWS_API_KEY = "93298ff83d9045a8ac42d4100b517856"
query = 'Adani'

start_date = (df.index.min() - pd.Timedelta(days=10)).date() # We go a few days eariler to avoid losing initial signal
end_date = datetime.today().date()

all_articles = []
current = start_date

while current <= end_date:
    next_date = min(current + timedelta(days=30), end_date)

    print(f"Fetching news from {current} to {next_date}")

    url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "from": current.isoformat(),
        "to": next_date.isoformat(),
        "language": "en",
        "sortBy": "publishedAt",
        "pageSize": 100,
        "apiKey": NEWS_API_KEY
    }

    response = requests.get(url, params=params)
    articles = response.json().get("articles", [])

    for art in articles:
        if art["title"] and art["publishedAt"]:
            all_articles.append({
                "headline": art["title"],
                "timestamp_utc": art["publishedAt"]
            })

    current = next_date + timedelta(days=1)

news_df = pd.DataFrame(all_articles)
news_df = pd.DataFrame(all_articles)

if len(news_df) == 0:
    print("⚠ No news fetched. Check API limits or date range.")
else:
    news_df['timestamp_utc'] = pd.to_datetime(news_df['timestamp_utc'], utc=True)

print("Total headlines:", len(news_df))

Fetching news from 2024-02-25 to 2024-03-26
Fetching news from 2024-03-27 to 2024-04-26
Fetching news from 2024-04-27 to 2024-05-27
Fetching news from 2024-05-28 to 2024-06-27
Fetching news from 2024-06-28 to 2024-07-28
Fetching news from 2024-07-29 to 2024-08-28
Fetching news from 2024-08-29 to 2024-09-28
Fetching news from 2024-09-29 to 2024-10-29
Fetching news from 2024-10-30 to 2024-11-29
Fetching news from 2024-11-30 to 2024-12-30
Fetching news from 2024-12-31 to 2025-01-30
Fetching news from 2025-01-31 to 2025-03-02
Fetching news from 2025-03-03 to 2025-04-02
Fetching news from 2025-04-03 to 2025-05-03
Fetching news from 2025-05-04 to 2025-06-03
Fetching news from 2025-06-04 to 2025-07-04
Fetching news from 2025-07-05 to 2025-08-04
Fetching news from 2025-08-05 to 2025-09-04
Fetching news from 2025-09-05 to 2025-10-05
Fetching news from 2025-10-06 to 2025-11-05
Fetching news from 2025-11-06 to 2025-12-06
Fetching news from 2025-12-07 to 2026-01-06
Fetching news from 2026-01-07 to

#### Now that we have obtained the data, we will align it according to the trade day

In [80]:
news_df['timestamp_utc'] = pd.to_datetime(news_df['timestamp_utc'], utc=True)
news_df['timestamp_ist'] = news_df['timestamp_utc'].dt.tz_convert(IST)
news_df['trading_day'] = news_df['timestamp_ist'].apply(map_to_trading_day)

news_df[['timestamp_ist','trading_day']].head()

,timestamp_ist,trading_day
0,2026-01-29 15:17:19+05:30,2026-01-29
1,2026-01-29 14:11:51+05:30,2026-01-29
2,2026-01-29 13:37:23+05:30,2026-01-29
3,2026-01-29 13:23:00+05:30,2026-01-29
4,2026-01-29 10:38:59+05:30,2026-01-29


#### Now Loading FinBERT to get sentiment on the headlines

In [66]:
def get_batch_sentiment(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    return (probs[:,0] - probs[:,1]).cpu().numpy()  # pos - neg

batch_size = 32
sentiments = []

for i in range(0, len(news_df), batch_size):
    batch_texts = news_df['headline'].iloc[i:i+batch_size].tolist()
    sentiments.extend(get_batch_sentiment(batch_texts))

news_df['sentiment'] = sentiments

#### Now aggregating daily headlines and their sentiment classification

In [67]:
daily_sentiment = news_df.groupby('trading_day')['sentiment'].mean()

df = df.merge(daily_sentiment, left_index=True, right_index=True, how='left')
df['sentiment'].fillna(0, inplace=True)

df[['sentiment']].head(20)

/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_82860/1082535320.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].fillna(0, inplace=True)


,sentiment
Date,
2024-03-07,0.0
2024-03-08,0.0
2024-03-11,0.0
2024-03-12,0.0
2024-03-13,0.0
2024-03-14,0.0
2024-03-15,0.0
2024-03-18,0.0
2024-03-19,0.0


#### Preparing the data to be fed into the LSTM

In [68]:
df['target'] = (df['log_return'].shift(-1) > 0).astype(int)
df.dropna(inplace=True)

features = ['log_return', 'rsi', 'macd', 'volatility', 'vol_chg', 'sentiment']

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [69]:
SEQ_LEN = 60

class MarketDataset(Dataset):
    def __init__(self, data, features):
        self.X = data[features].values
        self.y = data['target'].values

    def __len__(self):
        return len(self.X) - SEQ_LEN

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx:idx+SEQ_LEN], dtype=torch.float32),
            torch.tensor(self.y[idx+SEQ_LEN], dtype=torch.float32)
        )

dataset = MarketDataset(df, features)
train_size = int(0.8 * len(dataset))
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, len(dataset)-train_size])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

#### Making the LSTM Model for classification whether stock should move up or down on the next day

In [55]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return torch.sigmoid(self.fc(out))

In [70]:
def compute_ic(model, loader):
    model.eval()
    preds_all, targets_all = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).squeeze().cpu().numpy()
            preds_all.extend(preds)
            targets_all.extend(y_batch.numpy())

    return np.corrcoef(preds_all, targets_all)[0,1]

def train_model(hidden_dim, num_layers):
    model = LSTMClassifier(len(features), hidden_dim, num_layers).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(15):  # shorter training for grid search
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

    val_ic = compute_ic(model, val_loader)
    return model, val_ic

hidden_options = [32, 64, 128]
layer_options = [1, 2, 3]

results = []

for h in hidden_options:
    for l in layer_options:
        print(f"Training model: hidden={h}, layers={l}")
        model, ic = train_model(h, l)
        results.append((h, l, ic))
        print(f"Validation IC: {ic:.4f}\n")

Training model: hidden=32, layers=1


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Validation IC: 0.1235

Training model: hidden=32, layers=2
Validation IC: 0.1231

Training model: hidden=32, layers=3
Validation IC: 0.0704

Training model: hidden=64, layers=1
Validation IC: 0.1430

Training model: hidden=64, layers=2
Validation IC: 0.1408

Training model: hidden=64, layers=3
Validation IC: 0.1500

Training model: hidden=128, layers=1
Validation IC: 0.0944

Training model: hidden=128, layers=2
Validation IC: 0.1560

Training model: hidden=128, layers=3
Validation IC: 0.1805



In [71]:
results_df = pd.DataFrame(results, columns=['Hidden', 'Layers', 'IC'])
best_row = results_df.loc[results_df['IC'].idxmax()]

print("Best Architecture:")
print(best_row)

Best Architecture:
Hidden    128.000000
Layers      3.000000
IC          0.180477
Name: 8, dtype: float64


In [72]:
best_hidden = int(best_row['Hidden'])
best_layers = int(best_row['Layers'])

best_model = LSTMClassifier(len(features), best_hidden, best_layers).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=0.001)

for epoch in range(40):
    best_model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)

        optimizer.zero_grad()
        preds = best_model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(best_model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.6964
Epoch 2, Loss: 0.6932
Epoch 3, Loss: 0.6898
Epoch 4, Loss: 0.6880
Epoch 5, Loss: 0.6788
Epoch 6, Loss: 0.6797
Epoch 7, Loss: 0.6732
Epoch 8, Loss: 0.6709
Epoch 9, Loss: 0.6588
Epoch 10, Loss: 0.6575
Epoch 11, Loss: 0.6595
Epoch 12, Loss: 0.6530
Epoch 13, Loss: 0.6459
Epoch 14, Loss: 0.6395
Epoch 15, Loss: 0.6348
Epoch 16, Loss: 0.6267
Epoch 17, Loss: 0.6478
Epoch 18, Loss: 0.6197
Epoch 19, Loss: 0.6038
Epoch 20, Loss: 0.5961
Epoch 21, Loss: 0.6129
Epoch 22, Loss: 0.6079
Epoch 23, Loss: 0.6106
Epoch 24, Loss: 0.5930
Epoch 25, Loss: 0.5757
Epoch 26, Loss: 0.5548
Epoch 27, Loss: 0.5477
Epoch 28, Loss: 0.5582
Epoch 29, Loss: 0.5255
Epoch 30, Loss: 0.5256
Epoch 31, Loss: 0.4882
Epoch 32, Loss: 0.4910
Epoch 33, Loss: 0.4703
Epoch 34, Loss: 0.4548
Epoch 35, Loss: 0.4432
Epoch 36, Loss: 0.4282
Epoch 37, Loss: 0.4206
Epoch 38, Loss: 0.4058
Epoch 39, Loss: 0.3621
Epoch 40, Loss: 0.3387


In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, loader):
    model.eval()
    preds_all, targets_all = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).squeeze().cpu().numpy()
            preds_all.extend(preds)
            targets_all.extend(y_batch.numpy())

    preds_all = np.array(preds_all)
    targets_all = np.array(targets_all)

    # Convert probabilities → class labels
    pred_labels = (preds_all > 0.5).astype(int)

    acc = accuracy_score(targets_all, pred_labels)
    precision = precision_score(targets_all, pred_labels)
    recall = recall_score(targets_all, pred_labels)
    f1 = f1_score(targets_all, pred_labels)
    ic = np.corrcoef(preds_all, targets_all)[0,1]
    cm = confusion_matrix(targets_all, pred_labels)

    print("Model Evaluation")
    print("---------------------")
    print(f"Accuracy     : {acc:.4f}")
    print(f"Precision    : {precision:.4f}")
    print(f"Recall       : {recall:.4f}")
    print(f"F1 Score     : {f1:.4f}")
    print(f"Information Coefficient (IC): {ic:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    return acc, precision, recall, f1, ic

evaluate_model(best_model, val_loader)

Model Evaluation
---------------------
Accuracy     : 0.5476
Precision    : 0.5882
Recall       : 0.4545
F1 Score     : 0.5128
Information Coefficient (IC): 0.1224

Confusion Matrix:
[[26 14]
 [24 20]]


(0.5476190476190477,
 0.5882352941176471,
 0.45454545454545453,
 0.5128205128205128,
 0.12237172233308848)